In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from ortools.sat.python import cp_model
from constraint import Problem, AllDifferentConstraint

In [3]:
# Define constants 
DOCTORS = ['Doctor 1', 'Doctor 2']
SERVICES = {
    'Routine_Checkup': 7,  # 9am - 4pm
    'Blood_Test': 4,       # 9am - 1pm
    'Surgery': 5           # 10am - 3pm
}

# Define the weekly schedule 
WEEKDAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
SERVICE_DAYS = {
    'Routine_Checkup': ['Monday', 'Tuesday', 'Wednesday', 'Friday'],
    'Blood_Test': ['Wednesday'],
    'Surgery': ['Thursday']
}

# Define number of weeks 
NUM_WEEKS = 2  

# Initialize 
problem = Problem()

# Add variables for each service in each week, with domains of the doctors
for week in range(1, NUM_WEEKS + 1):
    for service in SERVICES.keys():
        for day in SERVICE_DAYS[service]:
            service_name = f"{service}_{day}_Week{week}"
            problem.addVariable(service_name, DOCTORS)

# Constraint 1: Balance the total hours worked for both doctors across all weeks
def balance_workload(*assigned_doctors):
    doctor_hours = {'Doctor 1': 0, 'Doctor 2': 0}
    idx = 0
    for service in SERVICES.keys():
        for day in SERVICE_DAYS[service]:
            for week in range(1, NUM_WEEKS + 1):
                service_name = f"{service}_{day}_Week{week}"
                doctor_hours[assigned_doctors[idx]] += SERVICES[service]
                idx += 1
    # Ensure both doctors have equal workload
    return doctor_hours['Doctor 1'] == doctor_hours['Doctor 2']


problem.addConstraint(balance_workload, [f"{service}_{day}_Week{week}" for service in SERVICES.keys() for day in SERVICE_DAYS[service] for week in range(1, NUM_WEEKS + 1)])

# Get solutions to the CSP
solutions = problem.getSolutions()

# Display solutions 
if solutions:
    solution = solutions[0]  # first solution
    print("Service Schedule")

    for service, days in SERVICE_DAYS.items():
        for day in days:
            for week in range(1, NUM_WEEKS + 1):
                service_name = f"{service}_{day}_Week{week}"
                if service_name in solution:
                    doctor = solution[service_name]
                    # Print the doctor assignments for each service, week, and day
                    if doctor == "Doctor 1":
                        print(f"Week{week}_{day}_{service.replace('_', ' ')}_Dr1:\t{SERVICES[service]}")
                        print(f"Week{week}_{day}_{service.replace('_', ' ')}_Dr2:\t0")
                    elif doctor == "Doctor 2":
                        print(f"Week{week}_{day}_{service.replace('_', ' ')}_Dr1:\t0")
                        print(f"Week{week}_{day}_{service.replace('_', ' ')}_Dr2:\t{SERVICES[service]}")
else:
    print("No solution found")

Service Schedule
Week1_Monday_Routine Checkup_Dr1:	0
Week1_Monday_Routine Checkup_Dr2:	7
Week2_Monday_Routine Checkup_Dr1:	0
Week2_Monday_Routine Checkup_Dr2:	7
Week1_Tuesday_Routine Checkup_Dr1:	7
Week1_Tuesday_Routine Checkup_Dr2:	0
Week2_Tuesday_Routine Checkup_Dr1:	7
Week2_Tuesday_Routine Checkup_Dr2:	0
Week1_Wednesday_Routine Checkup_Dr1:	7
Week1_Wednesday_Routine Checkup_Dr2:	0
Week2_Wednesday_Routine Checkup_Dr1:	7
Week2_Wednesday_Routine Checkup_Dr2:	0
Week1_Friday_Routine Checkup_Dr1:	0
Week1_Friday_Routine Checkup_Dr2:	7
Week2_Friday_Routine Checkup_Dr1:	0
Week2_Friday_Routine Checkup_Dr2:	7
Week1_Wednesday_Blood Test_Dr1:	0
Week1_Wednesday_Blood Test_Dr2:	4
Week2_Wednesday_Blood Test_Dr1:	4
Week2_Wednesday_Blood Test_Dr2:	0
Week1_Thursday_Surgery_Dr1:	0
Week1_Thursday_Surgery_Dr2:	5
Week2_Thursday_Surgery_Dr1:	5
Week2_Thursday_Surgery_Dr2:	0


In [4]:
number_of_solutions = len(solutions)
print(f"Number of possible solutions: {number_of_solutions}")

Number of possible solutions: 280
